In [2]:
# Install all The Required Libraries

from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


In [3]:
# Class to retrive ohlc data of the equity
class ScriptData:

    def __init__(self):
        self.data = {}
        self.ts = TimeSeries(key='MQEA4163N87R0BIO')
        
    # function to fetch the ohlc data and return the output as dictionary format
    def fetch_intraday_data(self,sym):
        df, meta_data = self.ts.get_intraday(sym)
        self.data[f'{sym}'] = df

        return df , sym

    # data preparation function to convert the dict ohlc data to pandas dataframe
    def convert_intraday_data(self,sym):
        df = self.data[sym]
        df = pd.DataFrame(df).transpose().reset_index()
        df.columns = ['timestamp','open','high','low','close','volume']

        # to convert the data types of our ohlc df
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['volume'] = pd.to_numeric(df['volume'])
        df['close'] = pd.to_numeric(df['close'])
        df['high'] = pd.to_numeric(df['high'])
        df['low'] = pd.to_numeric(df['low'])
        df['open'] = pd.to_numeric(df['open'])
        self.data[f'{sym}'] = df

        return df

    def __getitem__(self, key):
          
        return self.data[key]
    

# Indicator function to get the moving average of the close of our equity
def indicator1(df , timeperiod):

    df['indicator'] = np.nan

    for i in range(timeperiod-1, len(df)):
        df['indicator'][i] = sum(df['close'][i-timeperiod+1 : i+1])/timeperiod

    return df[['timestamp','indicator']].copy()




In [4]:
script = ScriptData()

In [6]:
script.fetch_intraday_data('NVDA')
df = script.convert_intraday_data('NVDA')
df


,timestamp,open,high,low,close,volume
0,2022-12-29 20:00:00,145.9900,145.9900,145.8800,145.950,4236
1,2022-12-29 19:45:00,146.0300,146.0500,145.8600,146.000,17837
2,2022-12-29 19:30:00,146.0100,146.0500,146.0100,146.020,3954
3,2022-12-29 19:15:00,146.0200,146.0500,146.0100,146.010,5799
4,2022-12-29 19:00:00,146.0900,146.1000,146.0500,146.050,963
...,...,...,...,...,...,...
95,2022-12-28 12:15:00,140.0300,140.3800,139.6615,139.910,776158
96,2022-12-28 12:00:00,140.3000,140.3400,139.5219,140.070,885953
97,2022-12-28 11:45:00,140.2900,140.6099,139.8600,140.300,830255
98,2022-12-28 11:30:00,139.6000,140.5300,139.4500,140.280,1315283


In [7]:

indicator1(df, timeperiod= 5)

,timestamp,indicator
0,2022-12-29 20:00:00,NaN
1,2022-12-29 19:45:00,NaN
2,2022-12-29 19:30:00,NaN
3,2022-12-29 19:15:00,NaN
4,2022-12-29 19:00:00,146.00600
...,...,...
95,2022-12-28 12:15:00,140.91458
96,2022-12-28 12:00:00,140.59902
97,2022-12-28 11:45:00,140.35102
98,2022-12-28 11:30:00,140.19426


In [8]:
# Class strategy to retrieve the buy and sell signals 
class Strategy:

    def __init__(self , symbol):
        self.symbol  = symbol
        

        self.df = pd.DataFrame()

    # Func to do the data preparation and retrieval of ohlc data and the Simple Moving Average Indicator
    def get_script_data(self):
        self.script = ScriptData()
        self.script.fetch_intraday_data(self.symbol)
        self.script.convert_intraday_data(self.symbol)

        self.df = self.script.data[self.symbol]

        self.indicator = indicator1(df , timeperiod=5)

        self.df['indicator_data'] = self.indicator['indicator']


    # Function to calculate the buy and sell signal of Our Strategy 
    def get_signals(self):
        df = self.df

        df['signal'] = np.nan
        for i in range(len(df)):
            if df.iloc[i-1]['close']>df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] < df.iloc[i]['indicator_data']:
                df['signal'][i] = 'BUY'
            elif df.iloc[i-1]['close'] < df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] > df.iloc[i]['indicator_data']:
                df['signal'][i] = 'SELL'

        self.df = df
        
        # returning only the timestamp which should have buy or sell signal
        return self.df[['timestamp','signal']].copy().dropna()

    # Ploting Func
    def plot(self):
        df = self.df
        fig = px.line(df , y = ['close','indicator_data'])
        # plotting the crossover of close and Simple Moving Average
        fig.show()

        fig = go.Figure(data=[go.Candlestick(
                       open=df['open'], high=df['high'],
                       low=df['low'], close=df['close'] ) ])
        fig.add_trace(go.Line(y=df['indicator_data']))
        # Plotting the OHLC Candles and Simple Moving Average
        fig.show()



    

In [9]:
strategy = Strategy('NVDA')

In [10]:
strategy.get_script_data()

In [11]:
strategy.get_signals()

,timestamp,signal
7,2022-12-29 18:15:00,BUY
8,2022-12-29 18:00:00,SELL
10,2022-12-29 17:30:00,BUY
12,2022-12-29 17:00:00,SELL
13,2022-12-29 16:45:00,BUY
14,2022-12-29 16:30:00,SELL
15,2022-12-29 16:15:00,BUY
17,2022-12-29 15:45:00,SELL
19,2022-12-29 15:15:00,BUY
23,2022-12-29 14:15:00,SELL


In [12]:
strategy.plot()

c:\Users\anki2\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


